# Reconnaissance de la langue des signes

### Contexte du projet

Beaucoup de progrès et de recherches en IA ont été faites pour aider les personnes sourdes et muettes. L'apprentissage profond et la vision par ordinateur peuvent également être utilisés pour avoir un impact sur cette cause.

Cela peut être très utile pour les personnes sourdes et muettes dans la communication avec les autres car la connaissance de la langue des signes n'est pas quelque chose qui est commun à tous, de plus, cela peut être étendu à la création des éditeurs automatiques, où la personne peut facilement écrire par ses simples gestes .


Nous utiliserons l'aphabet du langage des signes suivant:

![alphabet](alphabet.png)

Dans ce projet de reconnaissance de la langue des signes, nous créons un détecteur de signes, qui détecte les lettres de l'alphabet qui peuvent très facilement être étendus pour couvrir une vaste multitude d'autres signes et gestes de la main.

On va utiliser pour ce projet les modules OpenCV et Keras de python.


Le projet est divisé en 3 parties:

    1. Création du jeu de données
    2. Entraîner un CNN sur l'ensemble de données capturé
    3. Prédire les données



# Création du Dataset

Dans cette première partie, nous constituons un dataset avec les différents signes de l'alphabet. Nous avons choisi de collecter des images traitées en amont via un "filtre" (Thresholded) qui permettra de différencier au mieux chaque signe effectué avec la main.

Durant cette aquisition de donnée, nous chargeons préalablement le background, qui ne sera pas pris en compte lorsque nous positionnerons la main. Une fois fait, 300 images seront enregistrées dans un dossier conçu à cet effet. L'opération sera reproduite pour chaque catégorie (chaque signe de l'alphabet).

Le dataset constitué (près de 8100 images) sera compressé pour l'importer sur *Colab* afin d'entrainer notre modèle.

In [1]:
import cv2
import os
import time
import numpy as np

Il est assez possible d'obtenir l'ensemble de données dont nous avons besoin sur Internet, mais dans ce projet, 
nous créerons l'ensemble de données nous-mêmes.

Nous aurons un flux en direct de la caméra vidéo et chaque image qui détecte une main dans le ROI 
(région d'intérêt) créée sera enregistrée dans un répertoire (ici le répertoire dataset) qui contientra les 
répertoires de toutes les lettres de l'alphabet ainsi que l'espace (_SPACE)

Maintenant, pour créer l'ensemble de données, nous obtenons le flux de caméra en direct à l'aide d'OpenCV et 
créons un retour sur investissement qui n'est rien d'autre que la partie de l'image où nous voulons détecter 
la main pour les gestes.

La boîte rouge est le retour sur investissement et cette fenêtre sert à obtenir le flux de la caméra en direct 
de la webcam.

Pour différencier l'arrière-plan, nous calculons la moyenne pondérée accumulée pour l'arrière-plan, puis soustrayons celle-ci des cadres qui contiennent un objet devant l'arrière-plan qui peut être distingué comme premier plan.

Cela se fait en calculant le cumulated_weight pour certaines images (ici pour 60 images), nous calculons accumulated_avg pour l'arrière-plan.

Une fois que nous avons la moyenne accumulée pour l'arrière-plan, nous la soustrayons de chaque image que nous lisons après 60 images pour trouver tout objet qui couvre l'arrière-plan.


In [2]:
background = None
accumulated_weight = 0.9

# Création des dimensions du ROI

ROI_top = 50
ROI_bottom = 300
ROI_right = 50
ROI_left = 300

# Project: gesture-recognition, License: MIT License, OpenSource

def cal_accum_avg(frame, accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    
    cv2.accumulateWeighted(frame, background, accumulated_weight)

(Nous mettons en place un texte en utilisant cv2.putText pour afficher pour attendre et ne mettre aucun objet ou 
main dans le ROI lors de la détection de l'arrière-plan)

### Calculer la valeur seuil

Nous calculons maintenant la valeur de seuil pour chaque image et déterminons les contours à l'aide de 
cv2.findContours et nous renvoyons les contours max (les contours les plus extérieurs pour l'objet) à l'aide du 
segment de fonction. En utilisant les contours, nous sommes en mesure de déterminer s'il y a un objet de premier 
plan détecté dans le ROI, en d'autres termes, s'il y a une main dans le ROI.

In [3]:
# Fonction pour l'application du filtre (Thresholded)
def segment_hand(frame, threshold=20):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)
    thresholded = cv2.adaptiveThreshold(diff,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
    
    return (thresholded)

Lorsque des contours sont détectés (ou qu'une main est présente dans la ROI), nous commençons à enregistrer 
l'image de la ROI dans le répertoire "dataset" pour la lettre ou le chiffre pour lequel nous la détectons.

Ne pas oublier de créer au préalable un dossier pour chaque lettre de l'alphabet et _SPACE dans le répertoire "dataset".

In [38]:
# Ouverture de la caméra
cam = cv2.VideoCapture(0)

# Définition des variables (nombre de frame, dossier d'images)
num_frames = 0
element = '_SPACE'

num_imgs_taken = 0

while True:
    ret, frame = cam.read()
    
    # Eviter l'inversement de l'image
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]
    
    # Traitement
    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (5, 5), 0)
    
    if num_frames < 100:
        cal_accum_avg(gray_frame, accumulated_weight)
        
        # Détection du Background
        if num_frames <= 299:
            cv2.putText(frame_copy, "Chargement..", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
            
    # Configuration (Nous pouvons mettre notre main et la positionner correctement avant enregistrement)
    elif num_frames <= 300: 
        hand = segment_hand(gray_frame)
        cv2.putText(frame_copy, "Configuration.." + str(element), (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2)
        
        # Voir si la main est bien détectée
        if hand is not None:
            thresholded = hand
            
            # Dessiner les contours de la main
            cv2.putText(frame_copy, str(num_frames)+" Pour "+ str(element),(70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
            
            # Affiche l'image prise
            cv2.imshow("Image traitée", thresholded)
    
    else: 
        # Filtre sur la main
        hand = segment_hand(gray_frame)
        
        # Repérer si la main est là
        if hand is not None:
            
            thresholded = hand
            
            # Détection de la main (avec le filtre)
            cv2.putText(frame_copy, str(num_frames), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            cv2.putText(frame_copy, str(num_imgs_taken) + 'Enregistrement pour ' + str(element), (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            
            # Enregistrement des images de la main traitée dans la frame Filtre
            cv2.imshow("Image traitée", thresholded)
            if num_imgs_taken <= 299:
                cv2.imwrite(r"C:\\Users\\utilisateur\\Google Drive\\microsoft_ia\\Reconnaissance de la langue des signes\\projetfinal\\dataset\\"+str(element)+"\\"+str(num_imgs_taken+1) +'.jpg', thresholded)
                
            else:
                break
            num_imgs_taken +=1
            
        else:
            cv2.putText(frame_copy, 'Pas de main détectée !', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
    # Dessine une ROI de la frame Filtre
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right,ROI_bottom), (255,128,0), 3)
    
    # Incrementation
    num_frames += 1
    # Montre la frame Filtre
    cv2.imshow("Filtre", frame_copy)
    # Ferme l'application
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()
cam.release()